<a href="https://colab.research.google.com/github/lingchm/datascience/blob/master/exercises/socially_distanced_robots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression with Positivity Constraints

*Convex optimization*, *Projected gradient descent*, *primal-dual algorithm*

**Problem**

Given the least squares with positivity constraints:
\begin{equation}
\min_{x \in \mathbb{R}^N} \frac{1}{2} ||y-Ax||_2^2 \: \: \: \text{s.t.} x \geq 0,
\end{equation}
where $A \in \mathbb{R}^{MxN}$ and assume $rank(A) = N$.

This is natural in many practical applications where the entries of $x$ do not make sense where they are negative (e.g., light intensity, concentration of some physical material, etc.). 

**Method**

We explore three alternative approaches for solving this problem. 
1. One way is to write the Lagrangian function and directly solve the problem. 
2. Another way is using projected gradient descent. 
3. A third way is using the primal-dual approach, which avoids having to worry about step sizes. 

We implement all three algorithms below. 

**References**

Credits to Dr. Justin Romberg for designing this problem.


In [1]:
import numpy as np
import cvxpy as cp

ModuleNotFoundError: No module named 'cvxpy'

In [2]:
# Create data
m = 30
n = 10
np.random.seed(20212)
A = np.random.randn(m, n)
y = np.random.randn(m)

In [30]:
# Solution 1: CVXPT
x = cp.Variable(n)
objective = cp.Minimize(0.5*cp.norm(A@x - y,2)**2)
constraints = [0 <= x]
prob = cp.Problem(objective, constraints)

result = prob.solve()
xstar = x.value
lambdastar = constraints[0].dual_value
    
# verify optimality condition
print(A.T @ A @ xstar - A.T @ y - lambdastar)

In [3]:
# Solution 2: probjected gradient descent
def obj_function(A, x, y):
    return 0.5*np.linalg.norm(y - A @ x)**2
    
def gradf(A, x, y):
    return A.T @ A @ x - A.T @ y 

def prox(z):
    z[np.where(z < 0)] = 0
    return z
    
def projectedGD(x0, A, y, alpha=0.001, tol=1e-6, max_iter=10000):
    k, xk, xk_, ak = 0, x0, x0+1, alpha
    while np.linalg.norm(xk_ - xk) > tol and k < max_iter:
        # gradient step
        dk = -gradf(A, xk, y) 
        # projection step 
        xk_ = xk*1
        xk = prox(xk + ak * dk)
        k += 1
    print("Number of iterations of gradient descent:", k)
    print("Estimated x:", xk)
    return xk

In [4]:
print(np.linalg.norm(1/ (A.T @ A)))
projectedGD(np.zeros(n), A, y)


13.009669807351846
Number of iterations of gradient descent: 505
Estimated x: [0.         0.02932242 0.         0.         0.0357236  0.
 0.         0.         0.32954082 0.        ]


array([0.        , 0.02932242, 0.        , 0.        , 0.0357236 ,
       0.        , 0.        , 0.        , 0.32954082, 0.        ])

In [7]:
# Solution 3: primal-dual
def primaldual(x0, A, y, tol=1e-6, max_iter=1000):
    k, xk, xk_, lmbda = 0, x0, x0+1, x0+1
    while np.linalg.norm(xk_ - xk) > tol and k < max_iter:
        # primal
        xk_ = xk*1
        xk = np.linalg.inv(A.T @ A) @ (A.T @ y + lmbda)
        xk = prox(xk)
        # dual
        lmbda = (A.T @ A @ xk - A.T @ y)
        lmbda = prox(lmbda)
        # check complementary slackness
        lmbda[np.where(xk > tol)] = 0
        k += 1
    print("Number of iterations of gradient descent:", k)
    print("Estimated x:", xk)
    return xk

In [8]:
primaldual(np.zeros(n), A, y)

Number of iterations of gradient descent: 1000
Estimated x: [2.68960998e+55 0.00000000e+00 5.24503831e+54 0.00000000e+00
 4.79619276e+54 0.00000000e+00 4.13017869e+55 1.84109759e+55
 2.38295296e+55 0.00000000e+00]


array([2.68960998e+55, 0.00000000e+00, 5.24503831e+54, 0.00000000e+00,
       4.79619276e+54, 0.00000000e+00, 4.13017869e+55, 1.84109759e+55,
       2.38295296e+55, 0.00000000e+00])